In [1]:
#!pip install pandas

In [118]:
import pandas as pd
import re
import pandas as pd
import plotly.express as px
import plotly.express as px
import spacy
import numpy as np
import gensim.corpora as corpora
import gensim
import pyLDAvis.gensim_models
import pickle 
import pyLDAvis
import os
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

def import_chat(file):
    # Read the file
    with open(file, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    # Extract the data using regular expressions
    data = []
    for line in lines:
        match = re.findall('\[(.*?)\] (.*?): (.*)', line.strip())
        if match:
            data.append(match[0])
        
    # Create a pandas dataframe from the data
    print()
    df = pd.DataFrame(data, columns=['DateTime', 'Author', 'Message'])
    df.drop([0, 1, 2], axis='index', inplace=True)
    # Create a new dataframe with only valid rows
    valid_rows = []
    for i, row in df.iterrows():
        try:
            date_time = pd.to_datetime(row['DateTime'], format='%m/%d/%y, %I:%M:%S %p')
            row['Date'] = date_time.date()
            row['Time'] = date_time.time()
            valid_rows.append(row)
        except ValueError as e:
            print(f"Error on row {i}: {e}")
    df = pd.DataFrame(valid_rows)
    df.reset_index(inplace=True, drop=True)
    return df# Display the first 5 rows of the valid dataframe

def plot_message(file):
    data = import_chat(file)
# Get the value counts for each author
    author_freq = data['Author'].value_counts()
# Get the top 10 authors by message count
    top_authors = author_freq.iloc[:10]
# Create a dataframe with the author and message count data
    data_messages = pd.DataFrame({'Author': top_authors.index, 'Message Count': top_authors.values})
# Create the plot using plotly.express
    fig = px.bar(data_messages, x='Message Count', y='Author', orientation='h',
                 labels={'Message Count': 'Message Count', 'Author': 'Author'},
                 title='Top 10 Authors by Message Count')
    fig.show()

def character_count(data):
    data = import_chat(file)
    data['Message Character Count'] = data['Message'].apply(lambda x: len(x))
    dataframe = data.copy()
# Calculate the total character count for each author
    author_character_counts = data.groupby('Author')['Message Character Count'].sum()
# Sort the authors by their total character count in descending order
    sorted_authors = author_character_counts.sort_values(ascending=False)
    sorted_authors = sorted_authors.iloc[:10]
# Create a dataframe with the author and message count data
    data_character = pd.DataFrame({'Author': sorted_authors.index, 'Character Count': sorted_authors.values})
# Create the plot using plotly.express
    fig = px.bar(data_character, x='Character Count', y='Author', orientation='h',
                 labels={'Character Count': 'Character Count', 'Author': 'Author'},
                 title='Top 10 Authors by Message Count')
    fig.show()
    
    return dataframe

def pie_chart(data):
# Calculate the total character count for each author
    data = character_count(data)
    author_character_counts = data.groupby('Author')['Message Character Count'].sum()
# Sort the authors by their total character count in descending order
    sorted_authors = author_character_counts.sort_values(ascending=False)
    top_authors = sorted_authors.iloc[:10]
# Calculate the sum of the character counts
    total_character_count = top_authors.sum()
# Calculate the percentage of character count for each author
    author_percentages = [(count / total_character_count) * 100 for count in top_authors]
# Calculate the total character count of the authors outside the top 10
    other_authors = sorted_authors.iloc[10:]
    other_character_count = other_authors.sum()
# Calculate the percentage of character count for the other authors
    other_percentage = (other_character_count / total_character_count) * 100
# Create a dataframe with the author and percentage data, including 'others'
    data = pd.DataFrame({'Author': list(top_authors.index) + ['Others'], 'Percentage': list(author_percentages) + [other_percentage]})
# Create the plot using plotly.express
    fig = px.pie(data, values='Percentage', names='Author',
                 title='Top 10 Authors by Character Count (including Others)')
    fig.show()

def line_by_hour(data):
    data = character_count(data)
    data['DateTime'] = pd.to_datetime(data['DateTime'], format='%m/%d/%y, %I:%M:%S %p')
# Group the messages by hour
    hourly_counts = data.groupby(data['DateTime'].dt.hour)['Message'].count().reset_index(name='Message Count')
    hourly_counts['DateTime'] = hourly_counts['DateTime'].map({0: '12AM', 1: '1AM', 2: '2AM', 3: '3AM', 4: '4AM', 5: '5AM',
                                                               6: '6AM', 7: '7AM', 8: '8AM', 9: '9AM', 10: '10AM', 11: '11AM',
                                                               12: '12PM', 13: '1PM', 14: '2PM', 15: '3PM', 16: '4PM', 17: '5PM',
                                                               18: '6PM', 19: '7PM', 20: '8PM', 21: '9PM', 22: '10PM', 23: '11PM'})
# Plot the chart using plotly.express
    fig = px.line(hourly_counts, x='DateTime', y='Message Count', title='Messages by Hour')
    fig.show()


def pre_process(dataframe):
    pattern = r'[^a-zA-Z\s]|\s+[a-zA-Z]\s+|\s+'
# apply the regex pattern to the 'message' column, remove punctuations and emojis
    dataframe['New Message'] = dataframe['Message'].apply(lambda x: re.sub(pattern, ' ', x))

# convert the 'message' column to lowercase
    dataframe['New Message'] = dataframe['New Message'].str.lower()

    # Filter out the rows containing these texts
    dataframe = dataframe[~dataframe['Message'].str.contains('omitted|deleted')]
    filter_func = lambda s: ' '.join([w for w in s.split() if len(w) >= 6])
# apply the filter function to the specified column
    dataframe['New Message'] = dataframe['New Message'].apply(filter_func)
    dataframe['New Message'].replace('',np.nan,regex = True, inplace=True)
    dataframe.dropna(inplace=True)
    data_new = dataframe.loc[dataframe["New Message"].str.count(" ") >= 10]
    data_new.reset_index(inplace=True, drop=True)
    print('Preprocessing done')
    return data_new

def topic_modelling():
    stop_words = stopwords.words('english')
    stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'dey', 'go', 'na', 'people', 'even', 'make', 'know', 'one', 'still', 'like', 'say', 'time',
                       'sha', 'ooo', 'sure', 'first', 'person', 'want', 'wey', 'take', 'give', 'ni', 'day', 'problem', 
                       'Abeg',    'Oya',    'Nawa',    'Chop',    'Shishi',    'Wahala',    'Sabi',
                       'Jenifa',    'Soro',    'Kai',    'Banga',    'Wetin',    'Gbege',    'Kolo',    'Belle',    'Pikin',    'Gist',    'Ehen', 
                       'Shine',    'Oyinbo',    'Katakata',    'Hustle',    'Padi',    'Ikebe',    'Naija',    'Ojoro',    'Jollof',    'Jasi',   
                       'Waka',    'Ogbonge',    'Kpatakpata',    'Mumu',    'Orobo',    'Skelewu',    'Amebo',    'Aproko',    'Sisi',    'Yawa',  
                       'Chai',    'Abi',    'Gbese',    'Gbera',    'Go-slow',    'Gbe body e',    'Kpanlogo',    'Lai-lai',
                       'Palava',    'Suo',    'Totori',    'Zobo'])
    def sent_to_words(sentences):
        for sentence in sentences:
            # deacc=True removes punctuations
            yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
    def remove_stopwords(texts):
        return [[word for word in simple_preprocess(str(doc)) 
                 if word not in stop_words] for doc in texts]
    data = df['New Message'].tolist()
    data_words = list(sent_to_words(data))
    # remove stop words
    data_words = remove_stopwords(data_words)
    #print(data_words[:1][0][:30])
# Create Dictionary
    id2word = corpora.Dictionary(data_words)
# Create Corpus
    texts = data_words
    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]
# View
    num_topics = 10
# Build LDA model
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics)
    doc_lda = lda_model[corpus]
    def get_topic_keywords(lda_model):
        topic_keywords = {}
        for topic_id in range(lda_model.num_topics):
            topic_keywords[topic_id] = [word for word, prob in lda_model.show_topic(topic_id)]
        print(topic_keywords)
        return topic_keywords
# Visualize the topics
    # Visualize the topics
    pyLDAvis.enable_notebook()
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    return LDAvis_prepared


<>:27: DeprecationWarning:

invalid escape sequence \[

<>:27: DeprecationWarning:

invalid escape sequence \[

C:\Users\Dee\AppData\Local\Temp\ipykernel_3276\2091786194.py:27: DeprecationWarning:

invalid escape sequence \[

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dee\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [137]:
file = "C:/Users/Dee/Desktop/Portfolio/Chat Analysis/WhatsApp Chat - Engr.2021 Alumni(FET)/_chat.txt"
data = import_chat(file)


Error on row 12670: time data 'Breaking' does not match format '%m/%d/%y, %I:%M:%S %p' (match)


In [138]:
data

,DateTime,Author,Message,Date,Time
0,"5/4/22, 10:28:20 AM",‪+234 903 410 3691‬,"NYSC REGISTRATION FOR 2022 Batch B, STREAM 1 W...",2022-05-04,10:28:20
1,"5/4/22, 10:30:15 AM",Samuel Alabo | Makyr,001!!!!,2022-05-04,10:30:15
2,"5/4/22, 10:31:27 AM",‪+234 903 410 3691‬,How far,2022-05-04,10:31:27
3,"5/4/22, 10:33:17 AM",‪+234 903 410 3691‬,"You know my name guy, everytime shouldn't be a...",2022-05-04,10:33:17
4,"5/4/22, 10:33:54 AM",Quadri,‎sticker omitted,2022-05-04,10:33:54
...,...,...,...,...,...
20379,"2/19/23, 11:16:50 AM",Gbogo,We can't blame God. We can only blame ourselves.,2023-02-19,11:16:50
20380,"2/19/23, 11:16:50 AM",Dee 🏡Aero Dee,Just go to settings and click chat then export,2023-02-19,11:16:50
20381,"2/19/23, 11:17:08 AM",‪+234 803 887 9196‬,Ú can blame Buhari and also his advisers,2023-02-19,11:17:08
20382,"2/19/23, 11:17:09 AM",‪+234 810 413 0059‬,Blame humans simple,2023-02-19,11:17:09


In [140]:
data['Author'] = data['Author'].str.replace('[^\w\s]+|\s+', '', regex=True)
data['Author'] = data['Author'].astype(str)
radio_list = data['Author'].unique()
tuple(radio_list)

<>:1: DeprecationWarning:

invalid escape sequence \w

<>:1: DeprecationWarning:

invalid escape sequence \w

C:\Users\Dee\AppData\Local\Temp\ipykernel_3276\3229005799.py:1: DeprecationWarning:

invalid escape sequence \w



('2349034103691',
 'SamuelAlaboMakyr',
 'Quadri',
 'JamoVillaBaba',
 '2348146864224',
 'AyoCivil',
 'Mama',
 'LateeYekini',
 'TeslimCyborg',
 '2348104130059',
 '2348125759082',
 'Asiyanbolatayo',
 'DVGlory',
 'DeeAeroDee',
 'Habeebulah',
 'Gold',
 '2348167564278',
 '2347039610666',
 '2349065425811',
 '2347036125979',
 'EgeOla',
 'Ridwan',
 'Kemi',
 'KyrexAERO',
 '2348139433878',
 '2349079605713',
 '2348026026177',
 '2348143768841',
 'BoluMSE',
 'Gbogo',
 'Abhey',
 '2348111966849',
 '2349077095333',
 'KunmiMech',
 'Winnifred2',
 '2349029946259',
 '2347068878478',
 '2347032737892',
 '2348139426301',
 'FrancisAdedeji',
 '2347036248418',
 '2349054703681',
 '2348156165537',
 '2348159206302',
 'BolajiSmall',
 'DurosinlorunAkeem',
 'Walex',
 '2348156119190',
 '2348100158195',
 '2347018069372',
 'BoogieMan',
 'OlaMse',
 '2348135426805',
 'Engr2021AlumniFET',
 '2348026610695',
 '2347035800917',
 'ALSHShenko',
 'Afronasent',
 'Taiwo',
 'SulaMania',
 '2348140851696',
 'Westbrown',
 'AyodejiA',
 '

In [7]:
df['DateTime'] = pd.to_datetime(df['DateTime'], format='%m/%d/%y, %I:%M:%S %p')

# Group the messages by hour
hourly_counts = df.groupby(df['DateTime'].dt.hour).size().reset_index(name='Message Count')

# Plot the chart using plotly.express
fig = px.line(hourly_counts, x='DateTime', y='Message Count', title='Messages by Hour')
fig.show()

In [9]:
#!pip install spacy

In [10]:
#!python -m spacy download en_core_web_sm

Preprocessing done


[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [75]:
topic_modelling()

['wallohi', 'islamic', 'country', 'judgement', 'carried', 'muslim', 'leader', 'country', 'admonish', 'islamic', 'country', 'report', 'ignorance', 'criminal', 'extent', 'burning', 'forbidden', 'anybody', 'purnish', 'someone', 'except', 'interest', 'defense', 'ignorance']
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 3), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 2), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1)]
[(0,
  '0.012*"nigeria" + 0.012*"protest" + 0.012*"buhari" + 0.010*"government" + '
  '0.010*"country" + 0.010*"enough" + 0.008*"fulani" + 0.008*"remain" + '
  '0.008*"arrested" + 0.008*"nirsal"'),
 (1,
  '0.013*"system" + 0.012*"matter" + 0.012*"mistake" + 0.012*"someone" + '
  '0.012*"country" + 0.008*"checks" + 0.008*"dollars" + 0.008*"studies" + '
  '0.008*"memory" + 0.008*"months"'),
 (2,
  '0.029*"actually" + 0.021*"tinubu" + 0.013*"concerned" + 0.013*"nigeria" + '
  '0.009*"saying" + 0.009*"campaign" + 0.009*"normal" + 0.009*"finished" + '
  '0.

C:\Users\Dee\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.



PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.127379  0.067461       1        1  13.312335
8     -0.085847 -0.083231       2        1  12.911702
0      0.067455  0.049796       3        1  12.481163
7      0.008098 -0.048492       4        1  11.611205
2      0.028194 -0.126209       5        1   9.505033
6      0.057550 -0.039139       6        1   9.436122
1     -0.105358  0.072961       7        1   8.498301
3     -0.081496  0.058744       8        1   7.935483
4     -0.010262  0.031783       9        1   7.433734
5     -0.005713  0.016326      10        1   6.874923, topic_info=         Term       Freq      Total Category  logprob  loglift
397  actually   6.000000   6.000000  Default  30.0000  30.0000
595    nobody   3.000000   3.000000  Default  29.0000  29.0000
597    sacked   3.000000   3.000000  Default  28.0000  28.0000
382  governor   5.000000   5.000000  Default  27.0000  27.0000
179    tinubu  18.000000  18.000000  Default  26.0000  26.0000
..        ...        ...        ...      ...      ...      ...
84     health   0.610000   1.729602  Topic10  -5.1998   1.6351
11   interest   0.609998   3.164729  Topic10  -5.1998   1.0309
890   purpose   0.609998   1.883781  Topic10  -5.1998   1.5497
90     system   0.609996   6.754258  Topic10  -5.1998   0.2728
525   another   0.609995   2.335869  Topic10  -5.1998   1.3346

[531 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
694       5  0.801104     ability
787       8  0.770307      abroad
543       9  0.860381    accepted
66        1  0.513192   activated
397       2  0.158185    actually
...     ...       ...         ...
477       2  0.770890     whoever
636       5  0.801088     working
199       9  0.860377     youruba
877       1  0.775845      zenith
681       1  0.775840  zuckerberg

[632 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 9, 1, 8, 3, 7, 2, 4, 5, 6])

In [45]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'dey', 'go', 'na', 'people', 'even', 'make', 'know', 'one', 'still', 'like', 'say', 'time',
                   'sha', 'ooo', 'sure', 'first', 'person', 'want', 'wey', 'take', 'give', 'ni', 'day', 'problem', 
                   'Abeg',    'Oya',    'Nawa',    'Chop',    'Shishi',    'Wahala',    'Sabi',
                   'Jenifa',    'Soro',    'Kai',    'Banga',    'Wetin',    'Gbege',    'Kolo',    'Belle',    'Pikin',    'Gist',    'Ehen', 
                   'Shine',    'Oyinbo',    'Katakata',    'Hustle',    'Padi',    'Ikebe',    'Naija',    'Ojoro',    'Jollof',    'Jasi',   
                   'Waka',    'Ogbonge',    'Kpatakpata',    'Mumu',    'Orobo',    'Skelewu',    'Amebo',    'Aproko',    'Sisi',    'Yawa',  
                   'Chai',    'Abi',    'Gbese',    'Gbera',    'Go-slow',    'Gbe body e',    'Kpanlogo',    'Lai-lai',
                   'Palava',    'Suo',    'Totori',    'Zobo'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = df['New Message'].tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

['wallohi', 'islamic', 'country', 'judgement', 'carried', 'muslim', 'leader', 'country', 'admonish', 'islamic', 'country', 'report', 'ignorance', 'criminal', 'extent', 'burning', 'forbidden', 'anybody', 'purnish', 'someone', 'except', 'interest', 'defense', 'ignorance']


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dee\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [46]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 3), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 2), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1)]


In [69]:
from pprint import pprint
# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())

doc_lda = lda_model[corpus]

def get_topic_keywords(lda_model):
    topic_keywords = {}
    for topic_id in range(lda_model.num_topics):
        topic_keywords[topic_id] = [word for word, prob in lda_model.show_topic(topic_id)]
    print(topic_keywords)
    return topic_keywords

# Visualize the topics


[(0,
  '0.018*"enough" + 0.014*"grammar" + 0.011*"interest" + 0.009*"government" + '
  '0.009*"believe" + 0.009*"senate" + 0.009*"country" + 0.009*"comprehensive" '
  '+ 0.009*"purpose" + 0.009*"nigerians"'),
 (1,
  '0.028*"tinubu" + 0.026*"country" + 0.013*"system" + 0.012*"buhari" + '
  '0.011*"individual" + 0.010*"corruption" + 0.008*"except" + '
  '0.008*"judgement" + 0.008*"politics" + 0.008*"leaders"'),
 (2,
  '0.022*"buhari" + 0.019*"nigeria" + 0.016*"nobody" + 0.015*"sacked" + '
  '0.010*"president" + 0.010*"senior" + 0.010*"manifesto" + 0.010*"commission" '
  '+ 0.010*"paying" + 0.010*"tinubu"'),
 (3,
  '0.013*"president" + 0.011*"buhari" + 0.011*"friend" + 0.010*"transfers" + '
  '0.009*"general" + 0.009*"better" + 0.009*"anything" + 0.009*"surveillance" '
  '+ 0.009*"savings" + 0.009*"ikiootnomk"'),
 (4,
  '0.012*"business" + 0.012*"shepherd" + 0.012*"counterfeit" + '
  '0.012*"governor" + 0.006*"hiding" + 0.006*"million" + 0.006*"culture" + '
  '0.006*"almost" + 0.006*"amou

In [48]:
import pyLDAvis.gensim_models
import pickle 
import pyLDAvis
import os
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
LDAvis_prepared

C:\Users\Dee\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.



PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.086539  0.015763       1        1  15.498904
3      0.090883  0.065215       2        1  14.966420
7      0.132208 -0.112687       3        1  13.308657
1     -0.017997  0.006326       4        1  10.743260
5      0.028230  0.141104       5        1  10.320421
9     -0.053149 -0.046506       6        1   8.755953
6     -0.040449  0.010042       7        1   7.983771
4      0.044701 -0.014607       8        1   7.368557
0     -0.086568 -0.057429       9        1   6.749505
2     -0.011320 -0.007220      10        1   4.304552, topic_info=         Term       Freq      Total Category  logprob  loglift
4     country  19.000000  19.000000  Default  30.0000  30.0000
397  actually   6.000000   6.000000  Default  29.0000  29.0000
179    tinubu  19.000000  19.000000  Default  28.0000  28.0000
90     system   7.000000   7.000000  Default  27.0000  27.0000
920    prayer   3.000000   3.000000  Default  26.0000  26.0000
..        ...        ...        ...      ...      ...      ...
21     passed   0.436840   2.342030  Topic10  -5.0654   1.4663
121   believe   0.436838   8.311005  Topic10  -5.0654   0.1997
3     carried   0.436838   1.644506  Topic10  -5.0654   1.8199
404    normal   0.436838   2.242518  Topic10  -5.0654   1.5097
419   charges   0.436838   1.712217  Topic10  -5.0654   1.7795

[532 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
694       9  0.902312       ability
787       6  0.760407        abroad
281       7  0.818590     academics
337       9  0.902311        access
609       3  0.766932  accomplished
...     ...       ...           ...
181       8  0.832914        wisely
375       3  0.766935        within
314       4  0.527736     worldwide
170       2  0.506908        yoruba
170       5  0.506908        yoruba

[634 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 4, 8, 2, 6, 10, 7, 5, 1, 3])

In [ ]:
import numpy
print(numpy.__version__)

In [ ]:
df.describe()